In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk import Address, CodeMetadata, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ContractDeploymentBuilder, MultiESDTNFTTransferBuilder
from multiversx_sdk.core.transaction_builders import DefaultTransactionBuildersConfiguration
from utils.utils_tx import NetworkProviders
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id="D")
proxy = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")
network_provider = NetworkProviders("https://devnet-api.multiversx.com", proxy.url)

In [ ]:
contract_address = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqu6g8p2zaspelfee72kzht5ktynrs408fwn9qehq7ef")

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
owner = env.OWNER
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

In [ ]:
from utils.utils_chain import Account
from multiversx_sdk import AccountNonceHolder

token = "MYSTERYBOX-b466dd"
minter = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
user_nonce_holder = AccountNonceHolder(proxy.get_account(minter.address).nonce)

Send tokens

In [ ]:
import time

hashes = []

for i in range(50):
    user_account = Account(pem_file="~/Documents/split/C10.pem", pem_index=i%10)
    
    transfer = [
        TokenPayment.semi_fungible(token, 3, 1)
    ]

    builder = MultiESDTNFTTransferBuilder(
        config,
        destination=user_account.address,
        sender=minter.address,
        gas_limit=30000000,
        payments=transfer
    )

    tx = builder.build()
    tx.nonce = user_nonce_holder.get_nonce_then_increment()
    tx.signature = minter.signer.sign(tx)
    hash = proxy.send_transaction(tx)
    print(hash)
    hashes.append(hash)
    time.sleep(2)

logs processor

In [ ]:
from utils.utils_chain import decode_merged_attributes, base64_to_hex

def process_open_mystery_event(tx_hash):
    print(tx_hash)
    network_provider.wait_for_tx_executed(tx_hash)
    tx_data = proxy.get_transaction(tx_hash)

    if tx_data.status.is_failed():
        print("Transaction failed")
        return

    # print(tx_data.to_dictionary())
    event = tx_data.logs.find_first_or_none_event("openMysteryBox")
    if event is None:
        for sc_result in tx_data.contract_results.items:
            tx_data = proxy.get_transaction(sc_result.hash)
            event = tx_data.logs.find_first_or_none_event("openMysteryBox")
            if event is not None:
                break
    if event is None:
        print(f"No event found for: {tx_hash}")
        return
    event_data = base64_to_hex(event.data)
    decoded_data = decode_merged_attributes(event_data, 
                                            {
                                                'reward_type': 'u8',
                                                'reward_token_id': 'string',
                                                'value': 'biguint',
                                                'description': 'string',
                                                'percentage_chance': 'u64',
                                                'epochs_cooldown': 'u64'
                                            })
    print(decoded_data)

mystery box opener

In [ ]:
process_open_mystery_event("31e3c203c32ac4b84ef5dc6e8e53e97d4320fdec0d22cb6e56ec8ae942676446")

In [ ]:
import time
from multiversx_sdk import TransactionComputer

hashes = []
for i in range(30, 50):
    user_account = Account(pem_file="~/Documents/split/C10.pem", pem_index=i%10)
    
    transfer = [
        TokenPayment.semi_fungible(token, 3, 1)
    ]

    builder = ContractCallBuilder(
        config,
        contract=contract_address,
        function_name="openMysteryBox",
        caller=user_account.address,
        call_arguments=[],
        gas_limit=15000000,
        esdt_transfers=transfer
    )

    tx = builder.build()
    user_nonce_holder = AccountNonceHolder(proxy.get_account(user_account.address).nonce)
    tx.nonce = user_nonce_holder.get_nonce_then_increment()

    tx_computer = TransactionComputer()
    tx.signature = user_account.signer.sign(tx_computer.compute_bytes_for_signing(tx))
    hash = proxy.send_transaction(tx)
    hashes.append(hash)
    time.sleep(2)

for hash in hashes:
    process_open_mystery_event(hash)